In [2]:
# If not installed, run this cell. It is needed for watershed delineation
#!pip install pysheds
#!pip show pysheds

In [1]:
from pysheds.grid import Grid
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [27]:
# Read station catalogue file
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
csv_folder = "Streamflow_Stations"
csv_file = "Stations_Catalogue.csv"
csv_path = root_folder + "\\" + csv_folder + "\\" + csv_file
df_sts = pd.read_csv(csv_path)
df_sts

,ID,STATION,SUFIX,BASIN,LAT,LONG
0,07374525,USGS 07374525 Mississippi River at Belle Chass...,MISS,Mississippi,29.857145,-89.977850
1,23,Santiago,ULUA,Ulua,15.297001,-87.928217
2,17050001,Obidos,AMZN,Amazon,-1.947222,-55.511108
3,1662100,Dongola,NILE,Nile,19.186000,30.489500
4,2181900,Datong,YNGZ,Yangtze,30.770000,117.620000
5,A4261110,River Murray downstream Morgan (AMTD 316.5km),MUDA,Murray-Darling,-34.055059,139.685774
6,6742900,Ceatal Izmail,DABE,Danube,45.216667,28.716657
7,42055,Smederevo,DABE,Danube,44.666780,20.920600


In [28]:
# Assigning continent prefix to each station
df_sts["CONTINENT"] = ["na", "na", "sa", "af", "as", "au", "eu", "eu"]
df_sts

,ID,STATION,SUFIX,BASIN,LAT,LONG,CONTINENT
0,07374525,USGS 07374525 Mississippi River at Belle Chass...,MISS,Mississippi,29.857145,-89.977850,na
1,23,Santiago,ULUA,Ulua,15.297001,-87.928217,na
2,17050001,Obidos,AMZN,Amazon,-1.947222,-55.511108,sa
3,1662100,Dongola,NILE,Nile,19.186000,30.489500,af
4,2181900,Datong,YNGZ,Yangtze,30.770000,117.620000,as
5,A4261110,River Murray downstream Morgan (AMTD 316.5km),MUDA,Murray-Darling,-34.055059,139.685774,au
6,6742900,Ceatal Izmail,DABE,Danube,45.216667,28.716657,eu
7,42055,Smederevo,DABE,Danube,44.666780,20.920600,eu


In [29]:
hydrosheds = r"Z:\PhD_Datasets&Analysis\Info_Inputs\HydroSHEDS"
flow_dir_folder = hydrosheds + "\\FlowDirection"
flow_acc_folder = hydrosheds + "\\FlowAccumulation"

In [ ]:
# Specify directional mapping
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)

# Configuration of the plotting matrix
fig, axs = plt.subplots(4, 2, figsize=(10, 10))  # Create a 4x2 grid of subplots
fig.patch.set_alpha(0)
axs_ftn = axs.flatten()

pre_fdir = ""
pre_facc = ""

# Loop through the rows using iterrows()
for index, row in df_sts.iterrows():

    fdir_name = row['CONTINENT'] + "_dir_3s"
    if fdir_name != pre_fdir:
        print("Reading file: " + fdir_name +'.tif')
        grid = Grid.from_raster(flow_dir_folder + "\\" + fdir_name + "\\" + fdir_name +'.tif')
        fdir_grid = grid.read_raster(flow_dir_folder + "\\" + fdir_name + "\\" + fdir_name +'.tif')

    facc_name = row['CONTINENT'] + "_acc_3s"
    if facc_name != pre_facc:
        print("Reading file: " + facc_name +'.tif')
        facc_grid = grid.read_raster(flow_acc_folder + "\\" + facc_name + "\\" + facc_name +'.tif')

    x, y = row['LONG'], row['LAT']
    x_snap, y_snap = grid.snap_to_mask(facc_grid > 1000, (x, y))

    # Delineate the catchment
    catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir_grid, dirmap=dirmap, 
                       xytype='coordinate')
    
    # Clip the bounding box to the catchment
    grid.clip_to(catch)
    clipped_catch = grid.view(catch)

    # export to tiff
    grid.to_raster(clipped_catch, root_folder + "\\Streamflow_Sts_Drainage_Areas\\Cathment_" + row['BASIN'] + ".tif")

    axs_ftn[index].imshow(np.where(clipped_catch, clipped_catch, np.nan), extent=grid.extent, zorder=1, cmap='Greys_r')
    axs_ftn[index].set_title('Catchment + ' + row['BASIN'], size=10)
    axs_ftn[index].set_xlabel('Longitude')
    axs_ftn[index].set_ylabel('Latitude')
    axs_ftn[index].grid('on', zorder=0)

    pre_fdir = fdir_name
    pre_facc = facc_name

plt.tight_layout()  # Adjust subplots to fit into the figure area.
plt.show()